In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service  
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from bs4 import BeautifulSoup

s = Service('C:/Users/Corentin/Desktop/IA school/WEBscrapping/chromedriver-win64/chromedriver-win64/chromedriver.exe')
driver = webdriver.Chrome(service=s)

df_pages = {}

driver.get('https://www.schlouk-map.com/fr/cities/toulouse/happy-hour')

button = driver.find_element(By.CLASS_NAME, "fc-button-label")
time.sleep(1)
button.click()

html_content = '<p class="mb-4">66 bars</p>'
soup = BeautifulSoup(html_content, 'html.parser')
p_tag = soup.find('p', class_='mb-4')
nombres = p_tag.text.strip()
chiffres = re.findall(r'\d+', nombres)
chiffres_str = ''.join(chiffres)
chiffres_int = int(chiffres_str)

nb_pages = chiffres_int // 36 + 1
cpt = 2
liste_pages = [cpt + x for x in range(-1, nb_pages - 1)]

for page_num in liste_pages:
    driver.get(f'https://www.schlouk-map.com/fr/cities/toulouse/happy-hour?page={page_num}')
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    
    df_bar = pd.DataFrame(columns=['Nom', 'Adresse', 'Horaires', 'Services', 'Prix'])
    
    titles = soup.find_all("a", class_="name")
    adresses = soup.find_all("p", class_="text-truncate mb-0 text-secondary")

    for i, (title, adresse) in enumerate(zip(titles, adresses)):
        df_bar.loc[i, 'Nom'] = title.text.strip()
        df_bar.loc[i, 'Adresse'] = adresse.text.strip()

    hrefs = [f'https://www.schlouk-map.com{title["href"]}' for title in titles]

    big_liste_horaires = []
    big_liste_services = []
    big_liste_prices = []

    for href in hrefs:
        driver.get(href)
        time.sleep(1)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        table = soup.find('table', class_='table table-sm table-borderless mb-0')
        if table:
            table_texts = [cell.text.strip() for row in table.find_all('tr') for cell in row.find_all('td', recursive=False)]
            horaires = [f"{table_texts[i]} : {table_texts[i+1]}" for i in range(0, len(table_texts), 2)]
            big_liste_horaires.append(horaires)
        else:
            big_liste_horaires.append([])

        
        
        table_services = soup.find('table', class_='table table-sm table-borderless table-services mb-0')

        if table_services:
            services_text = [cell.text.strip() for row in table_services.find_all('tr') for cell in row.find_all('td', recursive=False)]
            concatenated_list = []

            for i in range(0, len(services_text)-1, 2):
                concatenated_string = services_text[i] + ":" + services_text[i+1]
                concatenated_list.append(concatenated_string)

            services = [element for element in concatenated_list if "Oui" in element]
            
            oui_parts = [element.split(":")[0] for element in services]
            print(oui_parts)
            big_liste_services.append(oui_parts)
        else:
            big_liste_services.append([])

        table_prices = soup.find('table', class_='table table-sm table-borderless table-drinks')
        if table_prices:
            prices_text = [cell.text.strip() for row in table_prices.find_all('tr') for cell in row.find_all('td', recursive=False)]
            prices_dict = {}
            for i in range(0, len(prices_text), 3):
                nom = prices_text[i]
                happyhour = prices_text[i+1]
                tarif_normal = prices_text[i+2]
                price_info = {
                    "nom": nom,
                    "happyhour": happyhour,
                    "tarif normal": tarif_normal
                }
                key = nom.lower() 
                prices_dict[key] = price_info
            big_liste_prices.append(prices_dict)
        else:
            big_liste_prices.append([])

    df_bar['Horaires'] = big_liste_horaires
    df_bar['Services'] = big_liste_services
    df_bar['Prix'] = big_liste_prices
    
    df_pages[page_num] = df_bar

df_final = pd.concat(df_pages.values(), ignore_index=True)


['Restauration', 'Vente à emporter', 'Wifi gratuit', '🌬 Air climatisé', 'Fléchettes', 'Jeux de société']
['Terrasse', 'Restauration', 'Vente à emporter', '🌬 Air climatisé', 'Accès & Toilettes Handicapés', 'DJ Mix', 'Chiens acceptés']
['Terrasse', 'Restauration', 'Diffusion de matchs', 'Fléchettes', 'Concerts / Musique Live']
['Terrasse', 'Restauration']
['Terrasse', 'Restauration', 'Concerts / Musique Live']
['Restauration', 'Concerts / Musique Live', 'DJ Mix']
['Terrasse', 'Restauration']
['Terrasse', 'Restauration', 'Vente à emporter', 'Wifi gratuit', 'Diffusion de matchs', '🌬 Air climatisé', 'Accès & Toilettes Handicapés', 'Concerts / Musique Live', 'DJ Mix', 'Chiens acceptés']
['Terrasse', 'Restauration', 'Diffusion de matchs', '🌬 Air climatisé', 'Fléchettes']
['Terrasse', 'Restauration', 'Diffusion de matchs', 'Concerts / Musique Live']
['Terrasse', 'Restauration', 'Diffusion de matchs', 'Concerts / Musique Live', 'DJ Mix']
['Terrasse', 'Restauration', 'Diffusion de matchs', 'Conc

In [4]:
print(big_liste_services)

[['Terrasse', 'Diffusion de matchs', 'Fléchettes'], ['Terrasse', 'Concerts / Musique Live'], ['Terrasse', 'Restauration', 'Concerts / Musique Live'], ['Terrasse', 'Restauration'], ['Restauration', 'Diffusion de matchs'], ['Terrasse', 'Restauration', 'Diffusion de matchs', 'Accès & Toilettes Handicapés', 'Concerts / Musique Live'], ['Terrasse', 'Restauration'], ['Terrasse', 'Restauration'], ['Terrasse', 'Restauration', 'Wifi gratuit', 'Diffusion de matchs', '🌬 Air climatisé'], ['Terrasse', 'Restauration', 'Diffusion de matchs', 'Fléchettes', 'Concerts / Musique Live'], ['Concerts / Musique Live', 'DJ Mix'], ['Restauration', 'Vente à emporter', 'Wifi gratuit', 'Chiens acceptés'], ['Terrasse', 'Restauration', 'Diffusion de matchs', '🌬 Air climatisé', 'Concerts / Musique Live', 'DJ Mix'], ['Terrasse', 'Restauration', 'Wifi gratuit', '🌬 Air climatisé', 'Accès & Toilettes Handicapés', 'Fléchettes', 'Concerts / Musique Live', 'DJ Mix'], ['Terrasse', 'Vente à emporter', 'Wifi gratuit', 'Diffus

In [5]:
for i, t in zip(range(len(df_bar)), range(len(big_liste_horaires))):
    df_final.loc[i, "Horaires"] = ", ".join(big_liste_horaires[t])
    df_final.loc[i, "Services"] = ", ".join(big_liste_services[t])
    df_final.loc[i, "Prix"] = str(big_liste_prices[t])
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

df_final




,Nom,Adresse,Horaires,Services,Prix
0,Matabiau Décapsule Club,"74 Rue Matabiau, 31000 Toulouse","Lundi : 17:00 - 00:00, Mardi : 17:00 - 00:00, Mercredi : 17:00 - 02:00, Jeudi : 17:00 - 02:00, Vendredi : 17:00 - 02:00, Samedi : 15:00 - 03:00, Dimanche : 15:00 - 00:00","Terrasse, Diffusion de matchs, Fléchettes","{'charlie wells(50cl)': {'nom': 'Charlie Wells(50cl)', 'happyhour': '5,20\xa0€', 'tarif normal': '-'}, 'bombardier(50cl)': {'nom': 'Bombardier(50cl)', 'happyhour': '5,50\xa0€', 'tarif normal': '-'}, 'aaaaaaaaaaaaaa': {'nom': 'aaaaaaaaaaaaaa', 'happyhour': '4,00\xa0€', 'tarif normal': '8,00\xa0€'}, 'aaaaaaaaaaa': {'nom': 'aaaaaaaaaaa', 'happyhour': '2,00\xa0€', 'tarif normal': '7,00\xa0€'}, 'aaaaaaaaaaaaaaaaa': {'nom': 'aaaaaaaaaaaaaaaaa', 'happyhour': '1,00\xa0€', 'tarif normal': '3,00\xa0€'}, 'aaaaaaaaaaaaaaaa': {'nom': 'aaaaaaaaaaaaaaaa', 'happyhour': '8,00\xa0€', 'tarif normal': '2,00\xa0€'}, 'aaaaaaaaaaaaa': {'nom': 'aaaaaaaaaaaaa', 'happyhour': '1,00\xa0€', 'tarif normal': '5,00\xa0€'}, 'aaaaaaaaaaaa': {'nom': 'aaaaaaaaaaaa', 'happyhour': '3,00\xa0€', 'tarif normal': '3,00\xa0€'}}"
1,Tom Pouce,"4 Place Roger Salengro, 31000 Toulouse","Lundi : 18:00 - 02:00, Mardi : 18:00 - 02:00, Mercredi : 18:00 - 02:00, Jeudi : 18:00 - 02:00, Vendredi : 18:00 - 02:00, Samedi : 18:00 - 03:00, Dimanche : Fermé","Terrasse, Concerts / Musique Live","{'bouteille de planteur(2cl)': {'nom': 'Bouteille de planteur(2cl)', 'happyhour': '15,00\xa0€', 'tarif normal': '-'}, 'bière(50cl)': {'nom': 'Bière(50cl)', 'happyhour': '5,00\xa0€', 'tarif normal': '-'}, 'aaaaaaaaaaaaaaaa': {'nom': 'aaaaaaaaaaaaaaaa', 'happyhour': '1,00\xa0€', 'tarif normal': '2,00\xa0€'}, 'aaaaaaaaaaaaaaaaaaaaa': {'nom': 'aaaaaaaaaaaaaaaaaaaaa', 'happyhour': '6,00\xa0€', 'tarif normal': '3,00\xa0€'}, 'aaaaaaaaaaaaaaaaaa': {'nom': 'aaaaaaaaaaaaaaaaaa', 'happyhour': '4,00\xa0€', 'tarif normal': '2,00\xa0€'}, 'aaaaaaaaaaaaaaaaaaaa': {'nom': 'aaaaaaaaaaaaaaaaaaaa', 'happyhour': '4,00\xa0€', 'tarif normal': '8,00\xa0€'}, 'aaaaaaaaaaaaa': {'nom': 'aaaaaaaaaaaaa', 'happyhour': '4,00\xa0€', 'tarif normal': '6,00\xa0€'}, 'aaaaaaaaaaaaaaa': {'nom': 'aaaaaaaaaaaaaaa', 'happyhour': '2,00\xa0€', 'tarif normal': '2,00\xa0€'}, 'aaaaaaaaaaaaaaaaa': {'nom': 'aaaaaaaaaaaaaaaaa', 'happyhour': '5,00\xa0€', 'tarif normal': '2,00\xa0€'}}"
2,The Black Lion,"4 Allées Paul Feuga, 31000 Toulouse","Lundi : Fermé, Mardi : 20:00 - 02:00, Mercredi : 20:00 - 02:00, Jeudi : 20:00 - 02:00, Vendredi : 20:00 - 02:00, Samedi : 20:00 - 03:00, Dimanche : Fermé","Terrasse, Restauration, Concerts / Musique Live","{'bière(50cl)': {'nom': 'Bière(50cl)', 'happyhour': '5,00\xa0€', 'tarif normal': '-'}, 'cocktail': {'nom': 'Cocktail', 'happyhour': '5,00\xa0€', 'tarif normal': '-'}}"
3,Barricot,"8 Esp. Compans Caffarelli, 31000 Toulouse","Lundi : 08:00 - 02:00, Mardi : 08:00 - 02:00, Mercredi : 08:00 - 02:00, Jeudi : 08:00 - 02:00, Vendredi : 08:00 - 02:00, Samedi : 18:00 - 03:00, Dimanche : Fermé","Terrasse, Restauration","{'verre de vin(17cl)': {'nom': 'Verre de vin(17cl)', 'happyhour': '2,00\xa0€', 'tarif normal': '-'}, 'soft': {'nom': 'Soft', 'happyhour': '2,00\xa0€', 'tarif normal': '-'}}"
4,La Maison,"9 Rue Gabriel Pérri, 31000 Toulouse","Lundi : 10:30 - 02:00, Mardi : 10:30 - 02:00, Mercredi : 10:30 - 02:00, Jeudi : 10:30 - 02:00, Vendredi : 10:30 - 02:00, Samedi : 16:00 - 03:00, Dimanche : Fermé","Restauration, Diffusion de matchs","{'stella artois(50cl)': {'nom': 'Stella Artois(50cl)', 'happyhour': '3,50\xa0€', 'tarif normal': '-'}, 'bière spéciale(50cl)': {'nom': 'Bière spéciale(50cl)', 'happyhour': '5,00\xa0€', 'tarif normal': '-'}}"
5,El Circo,"61 Rue Pargaminières, 31000 Toulouse","Lundi : Fermé, Mardi : 17:30 - 01:00, Mercredi : 17:30 - 01:00, Jeudi : 17:30 - 02:00, Vendredi : 17:30 - 02:00, Samedi : Fermé, Dimanche : Fermé","Terrasse, Restauration, Diffusion de matchs, Accès & Toilettes Handicapés, Concerts / Musique Live","{'licorne(50cl)': {'nom': 'Licorne(50cl)', 'happyhour': '-', 'tarif normal': '

In [6]:
df = pd.read_csv("bars.csv")
df.columns

Index(['Nom', 'Adresse', 'Horaires', 'Services', 'Prix'], dtype='object')

In [7]:
df_final.to_csv('bars.csv', index=False)

In [8]:
from geopy.geocoders import Nominatim
import folium
from geopy.exc import GeocoderUnavailable

geolocator = Nominatim(user_agent="my_geocoder")

toulouse_location = geolocator.geocode("Toulouse, France")

m = folium.Map(location=[toulouse_location.latitude, toulouse_location.longitude], zoom_start=14)

for nom, adresse,horaires, prix in zip(df['Nom'], df['Adresse'], df['Horaires'], df['Prix']):
    try:
        location = geolocator.geocode(adresse)
        if location:
            popup_text = f'<div style="min-width: 200px;"><a href="{nom}" target="_blank">{adresse}</a><br>{horaires}<br><strong>Prix :</strong> {prix} €<br>'

            folium.Marker([location.latitude, location.longitude], popup=popup_text,
                          tooltip=nom).add_to(m)
    except GeocoderUnavailable:
        print("Le service de géocodage est indisponible. Veuillez réessayer plus tard.")

m.save('map.html')

In [9]:
m

In [4]:
from jinja2 import Environment, FileSystemLoader

env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('report_template.html')

Le template a été chargé avec succès.


In [8]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText


bars = []
sender = 'mailtrap@demomailtrap.com'
receivers = 'corentingilles81@gmail.com'
message = template

msg = MIMEMultipart()
msg['From'] = sender
msg['To'] = receivers
msg['Subject'] = 'Meilleurs Deals à Toulouse'

html_content = template.render(bars=bars)

msg.attach(MIMEText(html_content, 'html'))

with smtplib.SMTP('live.smtp.mailtrap.io' ,587) as server:
    server.starttls()
    server.login('api','b417609d72c97f7a61aa457d9c496647')
    server.send_message(msg)
    print("message envoyé")

print("message envoyé2")

message envoyé
message envoyé2
